In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# ================== Settings ==================
input_dir = r"C:\Users\kabde\Desktop\ITI project\Brain Tumor MRI Classification\BrainTumor\BrainTumor\brain_tumor_dataset"  # Change to your folder path
output_dir = r"augmented_dataset"  # Output folder for the new images

In [ ]:
# Class names (same as folder names)
classes = ["no", "yes"]

In [ ]:
# ================== Create output folder ==================
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for cls in classes:
    os.makedirs(os.path.join(output_dir, cls), exist_ok=True)

In [ ]:
# ================== Count images for each class ==================
counts = {}
for cls in classes:
    cls_folder = os.path.join(input_dir, cls)
    counts[cls] = len(os.listdir(cls_folder))

max_count = max(counts.values())  # Largest number (will balance other classes to this)

print("Number of images before balancing:", counts)

In [ ]:

# ================== Prepare Augmentation for the smaller class ==================
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate images
for cls in classes:
    cls_folder = os.path.join(input_dir, cls)
    save_folder = os.path.join(output_dir, cls)
    img_files = os.listdir(cls_folder)
    
    # Copy all original images
    for img_name in img_files:
        img = cv2.imread(os.path.join(cls_folder, img_name))
        img = cv2.resize(img, (224, 224))
        cv2.imwrite(os.path.join(save_folder, img_name), img)
    
    # If class size is smaller than max_count, add images using augmentation
    num_to_generate = max_count - len(img_files)
    if num_to_generate > 0:
        print(f"Generating {num_to_generate} augmented images for class '{cls}'")
        img_index = 0
        while num_to_generate > 0:
            for img_name in img_files:
                if num_to_generate <= 0:
                    break
                img = cv2.imread(os.path.join(cls_folder, img_name))
                if img is None:
                    continue
                img = cv2.resize(img, (224, 224))
                img_array = np.expand_dims(img, 0)
                
                aug_iter = datagen.flow(
                    img_array, 
                    batch_size=1,
                    save_to_dir=save_folder,
                    save_prefix="aug",
                    save_format="jpg"
                )
                next(aug_iter)
                num_to_generate -= 1
                img_index += 1

print("Classes have been balanced and images saved in:", output_dir)